# Holiday Manager
## Starting Files
1. JSON file, [holidays.json](https://stage3talent.brightspace.com/content/enforced/6899-G10-Data-Fundamentals-22-05/M07/assets/files/holidays.json?_&d2lSessionVal=Mf89mKiNBB928zdpy7uBH8zVp&ou=6899), seeds application with base of holidays
2. website: [Time and Date Holidays](timeanddate.com/holidays/us), web scrape holidays from there and preload, only pull concrete dates! 2 years previous, this year, and 2 years future
3. API: [Open Weather Map API](https://rapidapi.com/community/api/open-weather-map), show weather for the current period
## Requirements
1. Seed application with holidays from JSON file
2. Scrape and add holiday dates from past, future, and now from Time and Date holidays
3. Save all holidays in JSON, in the same format as the original holidays.json file
4. Show weather for the current time period

In [1]:
# all packages
import json
from dataclasses import dataclass, field
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import date
from datetime import timedelta

currentYear = 2022
city = "New York City, USA"
weatherAPIKey = "55e9611df7msh8d9a1794a95dd96p1ff35bjsn79b2ab7fe896"

In [46]:
# all classes and member function definitions:
class Holiday:
      
    def __init__(self,name,date):
        #Your Code Here
        self.name = name
        if not isinstance(date, datetime):
            raise TypeError("date must be set to an datetime")
        self.date = date        
    
    def __str__(self):
        # String output
        # Holiday output when printed. Holiday Name (Date)
        outputstr = (f"{self.name} ({self.date.strftime('%Y-%m-%d')})")
        return outputstr

    def dictionaryOut(self):
        # to turn this element into a dictionary, date needs to be formatted as a string
        dictOut = {}
        dictOut['name'] = self.name
        dictOut['date'] = self.date.strftime('%Y-%m-%d')
        return dictOut

class HolidayList:
    def __init__(self):
        self.innerHolidays = []

    def __repr__(self):
        returnList = list(map(lambda b: b.__str__(), self.innerHolidays))
        returnStr = "\n".join(returnList)
        return returnStr

    def __str__(self):
        returnList = list(map(lambda b: b.__str__(), self.innerHolidays))
        returnStr = "\n".join(returnList)
        return returnStr
   
    def addHoliday(self,holidayObj):
        # Make sure holidayObj is an Holiday Object by checking the type
        if not isinstance(holidayObj, Holiday):
            raise TypeError("entry must be a Holiday type object")
        # Use innerHolidays.append(holidayObj) to add holiday
        self.innerHolidays.append(holidayObj)
        # print to the user that you added a holiday, would do this but the output would be really crowded...
        # print(f"{holidayObj.__str__()} has been added to the list!")
    
    def findHoliday(self, HolidayName, Date):
        # Find Holiday in innerHolidays
        if HolidayName == None and Date == None:
            results = "not found! no search terms defined!"
        elif Date == None:
            results = list(filter(lambda a: (a.name == HolidayName), self.innerHolidays))
            # print(f"Searching for holidays with name {HolidayName}: {len(results)} results")
        elif HolidayName == None:
            if not isinstance(Date, datetime):
                raise TypeError("date must be set to an datetime")
            results = list(filter(lambda a: (a.date == Date), self.innerHolidays))
            # print(f"Searching for holidays with date {Date}: {len(results)} results")
        else:
            if not isinstance(Date, datetime):
                raise TypeError("date must be set to an datetime")
            results = list(filter(lambda a: (a.name == HolidayName) and (a.date == Date), self.innerHolidays))
            # print(f"Searching for holidays with date {Date} and name {HolidayName}: {len(results)} results")
        # Return Holiday
        return results

    def removeHoliday(self, HolidayName, Date):
        if not isinstance(Date, datetime):
            raise TypeError("date must be set to an datetime")
        # Find Holiday in innerHolidays by searching the name and date combination.
        search = self.findHoliday(HolidayName, Date)
        holidayObject = Holiday(HolidayName,Date)
        if len(search) == 0: print(f"this holiday was not found on {Date}, please try again")
        else:
            print(f"{search[0].__str__()} has been found in the list!")
        # remove the Holiday from innerHolidays
            self.innerHolidays = list(filter(lambda a: (a.name != HolidayName or a.date != Date), self.innerHolidays))
        # inform user you deleted the holiday
            print("this holiday has been removed")
    
    def read_json(self,filelocation):
        # Read in things from json file location
        with open(filelocation) as file:
            holidaysJSON = json.load(file)['holidays']
        for i in range(0, len(holidaysJSON)):
            datetimeJSON= datetime.strptime(holidaysJSON[i]['date'], '%Y-%m-%d')
            singleHoliday = Holiday(holidaysJSON[i]['name'], datetimeJSON)
        # Use addHoliday function to add holidays to inner list.
            self.addHoliday(singleHoliday)

    def save_to_json(self,filelocation):
        # Write out json file to selected file.
        listHolidayDictionaries = list(map(lambda b: b.dictionaryOut(), self.innerHolidays))
        fullOutputDictionary = {'holidays': listHolidayDictionaries}
        JSONFormatting = json.dumps(fullOutputDictionary, indent=2)
        with open(filelocation, "w") as file:
            file.write(JSONFormatting)

    def scrapeHolidays(self):
        # Scrape Holidays from https://www.timeanddate.com/holidays/us/ 
        # Remember, 2 previous years, current year, and 2  years into the future. 
        # You can scrape multiple years by adding year to the timeanddate URL. 
        # For example https://www.timeanddate.com/holidays/us/2022
        global currentYear
        for year in range(currentYear-2,currentYear+3):
            url = "https://www.timeanddate.com/holidays/us/{}?hol=33554809".format(year)
            response = requests.get(url).text
            soup = BeautifulSoup(response,'html.parser')

            # isolate just the holiday table body
            holidayTable = soup.find('table',attrs={'id':'holidays-table'}).find('tbody')

            # get a list of all the specific holidays, taking out the first one because it is not a holiday lol
            perHoliday = holidayTable.find_all('tr', attrs={'class':'showrow'})
            count = 0 # to make sure all the holidays have been counted

            # going through all the holidays and getting the info
            for i in range(0,len(perHoliday)):
                # getting the date, converting to datetime using strings
                dateRaw = perHoliday[i].find('th', attrs={'class':'nw'}).text
                dateString = (f"{dateRaw} {year}")
                dateObject = datetime.strptime(dateString, "%b %d %Y")
                # getting the name
                name = perHoliday[i].find('a').text
                # search to see if it's already in the list
                # Check to see if name and date of holiday is in innerHolidays array
                search = self.findHoliday(name, dateObject)
                # Add non-duplicates to innerHolidays
                if len(search) == 0: 
                    self.addHoliday(Holiday(name, dateObject))
                    count += 1
                else:
                    # print(f"{name} on {dateString} is already in the list!")
                    pass
        
        # print(f"{count} holidays have been added for the year {year}.")
        # Handle any exceptions.     

    def numHolidays(self):
        # Return the total number of holidays in innerHolidays
        holidayNum = len(self.innerHolidays)
        return holidayNum

    def filter_holidays_by_week(self, year, week_number):
        # Use a Lambda function to filter by week number and save this as holidays, use the filter on innerHolidays
        # Week number is part of the the Datetime object
        # Cast filter results as list
        # filter by year
        yearFilter = list(filter(lambda a: (a.date.year == year), self.innerHolidays))
        # filter by week number
        weekFilter = list(filter(lambda a: (a.date.isocalendar()[1] == week_number), yearFilter))
        # return your holidays
        return weekFilter

    def displayHolidaysInWeek(self, holidayList):
        # Use your filter_holidays_by_week to get list of holidays within a week as a parameter
        # Output formated holidays in the week. 
        # * Remember to use the holiday __str__ method.
        # i don't really understand this function, just use print to display?
        for i in holidayList: print(i)

    def getWeather(self,year,weekNum):
        global city
        global weatherAPIKey
        global currentYear
        # Convert weekNum to range between two days
        today = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)
        monday = datetime.strptime(f'{year} {weekNum} 1', '%G %V %u')
        timeDiff = today - monday
        # Use Try / Except to catch problems
        # the API only queries for current weather data, the historical data is not working
        if timeDiff.days < -14 or timeDiff.days > 0:
            print("This weather API only works for the current day and a forecast of 2 weeks.")
            print("There is no weather data for this time period. Please try again!")
            print(f"Current year: {currentYear}, Current week: {today.isocalendar()[1]}")
            return
        # Query API for weather in that week range
        url = "https://community-open-weather-map.p.rapidapi.com/forecast/daily"
        querystring = {"q":city,"lat":"35","lon":"139","cnt":"16","units":"metric or imperial"}
        headers = {
            "X-RapidAPI-Key": weatherAPIKey,
            "X-RapidAPI-Host": "community-open-weather-map.p.rapidapi.com"
        }
        weather = requests.request("GET", url, headers=headers, params=querystring).text
        # Format weather information and return weather string.
        weatherDictRaw = json.loads(weather)["list"]
        weatherDictDate = {}
        for i in range(0,len(weatherDictRaw)):
            delta = timedelta(days=i)
            day = today + delta
            weatherDictDate[day] = weatherDictRaw[i]['weather'][0]['main']
        # filter and display output for just the days in that week
        weatherDictWeek = {}
        for i in range(1,8):
            weekday = datetime.strptime(f'{year} {weekNum} {i}', '%G %V %u')
            if weekday in list(weatherDictDate.keys()):
                weatherDictWeek[weekday] = weatherDictDate[weekday]
            else:
                weatherDictWeek[weekday] = "no data"
        return weatherDictWeek

    def viewCurrentWeek(self):
        # Use the Datetime Module to look up current week and year
        today = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)
        year = today.year
        weekN = today.isocalendar()[1]
        # Use your filter_holidays_by_week function to get the list of holidays for the current week/year
        weekHolidays = self.filter_holidays_by_week(year, weekN)
        # Use your displayHolidaysInWeek function to display the holidays in the week
        # Ask user if they want to get the weather
        while True:
            weatherDisplay = str(input("Would you like to see this week's weather? [y/n]: "))
            if weatherDisplay == "y" or weatherDisplay == "n":
                break
            else: print("invalid input!")
        # If yes, use your getWeather function and display results
        print("These are the holidays for this week:")
        if weatherDisplay == "y":
            weatherData = self.getWeather(year,weekN)
            # match each holiday date to weather date and print in an f string
            for event in weekHolidays:
                weatherEvent = weatherData[event.date]
                print(f"{event} - {weatherEvent}")
        elif weatherDisplay == "n":
            self.displayHolidaysInWeek(weekHolidays)
            

In [244]:

# testing datetime conversions, timedelta for getWeather functionality and error catching
year = 2022
weekNum = 1
monday = datetime.strptime(f'{year} {weekNum} 1', '%G %V %u')
sunday = datetime.strptime(f'{year} {weekNum} 7', '%G %V %u')
today = datetime.today()
timeDiff = today - monday
print(timeDiff)
def works(timeDiff):
    if timeDiff.days < -14 or timeDiff.days > 0:
        print("This weather API only works for the current day and a forecast of 2 weeks.")
        print("There is no weather data for this time period. Please try again!")
        return
    print("return does not work")
works(timeDiff)

189 days, 1:26:44.665685
This weather API only works for the current day and a forecast of 2 weeks.
There is no weather data for this time period. Please try again!


In [279]:
# Testing weather data functionality, matching holidays to their weathers
z = a.getWeather(2022,28)
y = a.filter_holidays_by_week(2022,28)

today = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)
year = today.year
weekN = today.isocalendar()[1]
# Use your filter_holidays_by_week function to get the list of holidays for the current week/year
weekHolidays = y
# Use your displayHolidaysInWeek function to display the holidays in the week
# Ask user if they want to get the weather
while True:
    weatherDisplay = str(input("Would you like to see this week's weather? [y/n]: "))
    if weatherDisplay == "y" or weatherDisplay == "n":
        break
    else: print("invalid input!")
# If yes, use your getWeather function and display results
print("These are the holidays for this week:")
if weatherDisplay == "y":
    weatherData = z
    # match each holiday date to weather date and print in an f string
    for event in weekHolidays:
        weatherEvent = weatherData[event.date]
        print(f"{event} - {weatherEvent}")
elif weatherDisplay == "n":
    for i in y: print(i)

These are the holidays for this week:
Bastille Day (2022-07-14) - Rain
Rural Transit Day (2022-07-16) - Rain


In [47]:
# testing multiple year holiday removal
testNYE = HolidayList()
name = "New Year's Eve"
for year in range(2020, 2025):
    dateString = (f"Dec 31 {year}")
    dateObject = datetime.strptime(dateString, "%b %d %Y")
    holidayItem = Holiday(name, dateObject)
    testNYE.addHoliday(holidayItem)
name = "LAST DAY"
for year in range(2020, 2025):
    dateString = (f"Dec 31 {year}")
    dateObject = datetime.strptime(dateString, "%b %d %Y")
    holidayItem = Holiday(name, dateObject)
    testNYE.addHoliday(holidayItem)
name = "oh boy"
for year in range(2020, 2025):
    dateString = (f"Jan 1 {year}")
    dateObject = datetime.strptime(dateString, "%b %d %Y")
    holidayItem = Holiday(name, dateObject)
    testNYE.addHoliday(holidayItem)

print(testNYE)

# removing multiple holidays
dateObject1 = datetime.strptime("Dec 31 2020", "%b %d %Y")
testNYE.removeHoliday("New Year's Eve", dateObject1)

print(testNYE)

New Year's Eve (2020-12-31)
New Year's Eve (2021-12-31)
New Year's Eve (2022-12-31)
New Year's Eve (2023-12-31)
New Year's Eve (2024-12-31)
LAST DAY (2020-12-31)
LAST DAY (2021-12-31)
LAST DAY (2022-12-31)
LAST DAY (2023-12-31)
LAST DAY (2024-12-31)
oh boy (2020-01-01)
oh boy (2021-01-01)
oh boy (2022-01-01)
oh boy (2023-01-01)
oh boy (2024-01-01)
New Year's Eve (2020-12-31) has been found in the list!
this holiday has been removed
New Year's Eve (2021-12-31)
New Year's Eve (2022-12-31)
New Year's Eve (2023-12-31)
New Year's Eve (2024-12-31)
LAST DAY (2020-12-31)
LAST DAY (2021-12-31)
LAST DAY (2022-12-31)
LAST DAY (2023-12-31)
LAST DAY (2024-12-31)
oh boy (2020-01-01)
oh boy (2021-01-01)
oh boy (2022-01-01)
oh boy (2023-01-01)
oh boy (2024-01-01)


In [273]:
# testing member functions 
a = HolidayList()
a.read_json('test_holidayoutput.json')
a.scrapeHolidays()
a.numHolidays()

Margaret Thatcher Day (2021-01-10) has been added to the list!
World Sketchnote Day (2021-01-11) has been added to the list!
Zanzibar Revolution Day (2021-01-12) has been added to the list!
National Rubber Ducky Day (2021-01-13) has been added to the list!
Tamil Thai Pongdal Day (2021-01-14) has been added to the list!
National Bagel Day (2021-01-15) has been added to the list!
Signing of the Peace Accords (2021-01-16) has been added to the list!
New Year's Day (2020-01-01) has been added to the list!
Stephen Foster Memorial Day (2020-01-13) has been added to the list!
Martin Luther King Jr. Day (2020-01-20) has been added to the list!
Lunar New Year (2020-01-25) has been added to the list!
Kansas Day (2020-01-29) has been added to the list!
National Freedom Day (2020-02-01) has been added to the list!
First Day of Black History Month (2020-02-01) has been added to the list!
Groundhog Day (2020-02-02) has been added to the list!
Super Bowl (2020-02-02) has been added to the list!
Natio

627

In [222]:
# testing week filtering functionality

# how to properly display filtered by week list
# it's in a list, so you need to print individual object in the list
for i in a.filter_holidays_by_week(2021,2): print(i)
z = a.filter_holidays_by_week(2021,2)


World Sketchnote Day (2021-01-11)
Zanzibar Revolution Day (2021-01-12)
National Rubber Ducky Day (2021-01-13)
Tamil Thai Pongdal Day (2021-01-14)
National Bagel Day (2021-01-15)
Signing of the Peace Accords (2021-01-16)
Stephen Foster Memorial Day (2021-01-13)


In [ ]:
# Testing for saving to JSON file
a = HolidayList()
a.read_json('holidays.json')
z = a.innerHolidays

# converting objects in innerholidays to dictionary format
zDict = list(map(lambda b: b.dictionaryOut(), z))
l = {'holidays':zDict}

text = json.dumps(l, indent=2)

with open("test_holidayoutput.json", "w") as file:
    file.write(text)

In [186]:
# Testing dictionary conversion in Holiday object for use converting HolidayList to dictionary

dtconvert = datetime.strptime("2021-01-13", '%Y-%m-%d')
ree = Holiday("National Rubber Ducky Day", dtconvert)

ree.dictionaryOut()

{'name': 'National Rubber Ducky Day', 'date': '2021-01-13'}

In [215]:
# Testing JSON import before putting it in class definitino
# legacy

# importing all holidays from json file
# holidays.json starting location
presetHolidaysLoc = 'holidays.json'

# initializing holidays dictionary
holList = []

# reading in holidays.json
def readHolidaysJson():
    global holidays
    # Read dictionary file in from dictionary file location
    with open(presetHolidaysLoc) as file:
        holidays = json.load(file)['holidays']
    for i in range(0, len(holidays)):
        datetime_object = datetime.strptime(holidays[i]['date'], '%Y-%m-%d')
        print(datetime_object.year, datetime_object.isocalendar()[1])
        a = Holiday(holidays[i]['name'], datetime_object)
        holList.append(a)
        
readHolidaysJson()

2021 1
2021 2
2021 2
2021 2
2021 2
2021 2
2021 2


In [ ]:
# Testing find holiday functionality
dtconvert = datetime.strptime("2021-01-13", '%Y-%m-%d')
z = holList.findHoliday("National Rubber Ducky Day", dtconvert)
print(z)

In [ ]:
# Testing remove holiday functionality
dtconvert = datetime.strptime("2021-01-13", '%Y-%m-%d')
holList.removeHoliday("National Rubber Ducky Day", dtconvert)
print(holList)

In [25]:
# Testing for webscraping before putting into member function
# legacy

# main body for extracting (1) name of the holiday, (2) date of the holiday
# main body of content: in <table id"holidays-table"...
def TaDScrape(year):
    global holidayList
    url = "https://www.timeanddate.com/holidays/us/{}?hol=33554809".format(year)
    response = requests.get(url).text
    soup = BeautifulSoup(response,'html.parser')

    # isolate just the holiday table body
    holidayTable = soup.find('table',attrs={'id':'holidays-table'}).find('tbody')

    # get a list of all the specific holidays, taking out the first one because it is not a holiday lol
    perHoliday = holidayTable.find_all('tr', attrs={'class':'showrow'})
    count = 0 # to make sure all the holidays have been counted

    # going through all the holidays and getting the info
    for i in range(0,len(perHoliday)):
        # getting the date, converting to datetime using strings
        dateRaw = perHoliday[i].find('th', attrs={'class':'nw'}).text
        dateString = (f"{dateRaw} {year}")
        dateObject = datetime.strptime(dateString, "%b %d %Y")
        # getting the name
        name = perHoliday[i].find('a').text
        holidayList.append(Holiday(name, dateObject))
        count += 1
        
    print(f"There are {count} holidays on this page for the year {year}.")

# scraping stuff
# get soup output for webpage per url
# combined into the TaDScrape function
def getSOUP(url):
    response = requests.get(url).text
    output = BeautifulSoup(response,'html.parser')
    return output

# dates output in the website as Jan 1, so convert this to json version: YYYY-MM-DD
# combined into the TaDScrape function
def dateConv(dateStr, year):
    dateString = (f"{dateStr} {year}")
    dateObject = datetime.strptime(dateString, "%b %d %Y")
    try:
        return dateObject
    except:
        print("data scraping: check if date formatting is still the same, edit dateConv()")

In [26]:
# scraping from website, based on year
currentYear = 2022
for year in range(currentYear-2,currentYear+3): TaDScrape(year)


There are 124 holidays on this page for the year 2020.
There are 127 holidays on this page for the year 2021.
There are 125 holidays on this page for the year 2022.
There are 124 holidays on this page for the year 2023.
There are 121 holidays on this page for the year 2024.


In [230]:
# Testing for weather API, only forecast data?

# using the 16 day forecast option for rapidAPI
url = "https://community-open-weather-map.p.rapidapi.com/forecast/daily"

querystring = {"q":city,"lat":"35","lon":"139","cnt":"16","units":"metric or imperial"}

headers = {
	"X-RapidAPI-Key": weatherAPIKey,
	"X-RapidAPI-Host": "community-open-weather-map.p.rapidapi.com"
}

weather = requests.request("GET", url, headers=headers, params=querystring).text

In [288]:
# Testing matching up the raw weather data with dates using only today as the metric

today = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)

weatherDictRaw = json.loads(weather)["list"]
weatherDictforDisplay = {}
for i in range(0,len(weatherDictRaw)):
    delta = timedelta(days=i)
    day = today+delta
    weatherDictforDisplay[day] = weatherDictRaw[i]['weather'][0]['main']

print(list(weatherDictforDisplay.keys()))

year = 2022
weekNum = 28
weatherDictWeek = {}
for i in range(1,8):
    weekday = datetime.strptime(f'{year} {weekNum} {i}', '%G %V %u')
    if weekday in list(weatherDictforDisplay.keys()):
        weatherDictWeek[weekday] = weatherDictforDisplay[weekday]
    else:
        weatherDictWeek[weekday] = "no data"

print(weatherDictWeek)

[datetime.datetime(2022, 7, 11, 0, 0), datetime.datetime(2022, 7, 12, 0, 0), datetime.datetime(2022, 7, 13, 0, 0), datetime.datetime(2022, 7, 14, 0, 0), datetime.datetime(2022, 7, 15, 0, 0), datetime.datetime(2022, 7, 16, 0, 0), datetime.datetime(2022, 7, 17, 0, 0), datetime.datetime(2022, 7, 18, 0, 0), datetime.datetime(2022, 7, 19, 0, 0), datetime.datetime(2022, 7, 20, 0, 0), datetime.datetime(2022, 7, 21, 0, 0), datetime.datetime(2022, 7, 22, 0, 0), datetime.datetime(2022, 7, 23, 0, 0), datetime.datetime(2022, 7, 24, 0, 0), datetime.datetime(2022, 7, 25, 0, 0), datetime.datetime(2022, 7, 26, 0, 0)]
{datetime.datetime(2022, 7, 11, 0, 0): 'Clear', datetime.datetime(2022, 7, 12, 0, 0): 'Rain', datetime.datetime(2022, 7, 13, 0, 0): 'Rain', datetime.datetime(2022, 7, 14, 0, 0): 'Rain', datetime.datetime(2022, 7, 15, 0, 0): 'Rain', datetime.datetime(2022, 7, 16, 0, 0): 'Rain', datetime.datetime(2022, 7, 17, 0, 0): 'Rain'}


In [20]:
# definitions for functions for sample UI check
savedState = False
theList = HolidayList()
newfilelocation = "test_holiday.json"

def StartUp():
    global theList
    theList.read_json('holidays.json')
    theList.scrapeHolidays()
    totalNum = theList.numHolidays()
    print("Holiday Management")
    print("====================")
    print(f"There are {totalNum} holidays stored in the system")

def MainMenu():
    print("====================")
    print("====================")
    print("Holiday Menu")
    print("====================")
    print("1. Add a Holiday")
    print("2. Remove a Holiday")
    print("3. Save Holiday List")
    print("4. View Holidays")
    print("5. Exit")
    while True:
        nextPage = int(input("What would you like to do? Enter the number of the option needed: "))
        if nextPage not in range(1,6):
            print("Wrong input!")
        else:
            if nextPage == 1: AddHoliday()
            elif nextPage == 2: RemoveHoliday()
            elif nextPage == 3: SaveHolidayList()
            elif nextPage == 4: ViewHolidays()
            elif nextPage == 5: 
                exitstatus = Exit()
                if exitstatus == True: break

def AddHoliday():
    print("Add a Holiday")
    print("====================")
    global theList
    global savedState
    newHolidayName = str(input("Holiday: "))
    newHolidayDateRaw = str(input("Date (YYYY-MM-DD): "))
    while True:
        try:
            newHolidayDate = datetime.strptime(newHolidayDateRaw, '%Y-%m-%d')
            break
        except:
            print("Error: \nInvalid date. Please try again.")
            newHolidayDateRaw = str(input(f"Date for {newHolidayName} (YYYY-MM-DD): "))
    newHolidayObject = Holiday(newHolidayName,newHolidayDate)
    theList.addHoliday(newHolidayObject)
    print(f"Success:\n{newHolidayObject} has been added to the holiday list.")
    savedState = False

def RemoveHoliday():
    print("Remove a Holiday")
    print("====================")
    global theList
    global savedState
    while True:
        removeHolidayName = str(input("Holiday Name: "))
        if removeHolidayName == "0": 
            break
        allDates = theList.findHoliday(removeHolidayName,None)
        if len(allDates) != 0:
            for i in allDates:
                theList = theList.removeHoliday(i.name, i.date)
            print(f"Success:\n{removeHolidayName} has been removed from the holiday list.")
            savedState = False
            break
        else:
            print(f"Error: \n{removeHolidayName} not found. Please try again, or enter 0 to exit.")

def SaveHolidayList():
    print("Saving Holiday List")
    print("====================")
    global theList
    global savedState
    global newfilelocation
    while True:
        save = str(input("Are you sure you want to save your changes? [y/n]: "))
        if save == "y":
            theList.save_to_json(newfilelocation)
            savedState = True
            print(f"Success:\nYour changes have been saved.")
            break
        elif save == "n":
            print(f"Canceled:\nHoliday list file save canceled.")
            break
        else: print("Wrong input!")


def ViewHolidays():
    print("View Holidays")
    print("====================")
    global theList
    global currentYear

    #correct year loop
    while True:
        year = int(input("Which year? "))
        if year in range(2010,currentYear+3): break
        else: 
            print("This year is likely not supported, or is in the wrong format.")
            print(f"Please enter a year between 2010 and {currentYear+2}")

    #correct week loop
    while True:
        week = str(input("Which week? #[1-52, Leave blank for the current week]: "))
        if week == "": break
        elif int(week) in range(1,53): break
        else: 
            print("This is not a possible calendar week number within a year, or is in the wrong format.")
            print(f"Please enter a week number between 1 and 52, or leave the input blank.")

    # a week input is not given: run view current week function
    if week == "":
        theList.viewCurrentWeek()
    elif int(week) in range(1,53):
        print(f"These are the holidays for {year} week #{week}:")
        displayList = theList.filter_holidays_by_week(year, int(week))
        theList.displayHolidaysInWeek(displayList)


def Exit():
    global savedState
    print("Exit")
    print("====================")
    while True:
        if savedState:
            exitStatus = str(input("Are you sure you want to exit? [y/n]: "))
        else: 
            exitStatus = str(input("Are you sure you want to exit?\nYour changes will be lost.\n[y/n]: "))
        if exitStatus == "y" or exitStatus == "n":
            break
        print("Wrong input!")
    if exitStatus == "y":
        # end program
        return True

StartUp()
MainMenu()

Holiday Management
There are 627 holidays stored in the system
Holiday Menu
1. Add a Holiday
2. Remove a Holiday
3. Save Holiday List
4. View Holidays
5. Exit
Add a Holiday
Success:
Hermione (2022-11-07) has been added to the holiday list.
Exit
